In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
import pandas as pd
import itertools
import sys
import os
import math
sys.path.append('../..')
import scipy.optimize as opt
from functools import partial
from hyperopt import hp, tpe, fmin, Trials
from models.optim.grad_opt import SEIR

In [2]:
# np.array([start days]), np.array([duration]), np.array([choice], number_of_days_simulated)
def run_seir(days, int_vec):
    R0 = 2.2 
    T_inf = 2.9
    T_trans = T_inf/R0
    T_inc = 5.2
    T_recov_mild = (14 - T_inf)
    T_hosp = 5
    T_recov_severe = (31.5 - T_inf)
    T_death = 32

    P_severe = 0.2
    P_fatal = 0.02
    P_mild = 1 - P_severe - P_fatal

    N = 7e6
    I0 = 1.0
    
    params = [T_trans, T_inc, T_inf, T_recov_mild, T_hosp, T_recov_severe, T_death, 
              P_mild, P_severe, P_fatal, N, int_vec]

    # S, E, I, R_mild, R_severe, R_severe_home, R_fatal, C, D
    state_init_values = [(N - I0)/N, 0, I0/N, 0, 0, 0, 0, 0, 0]
    
    solver = SEIR(params, state_init_values)
    sol = solver.solve_ode(time_step=1, total_no_of_days=days)
    states_int_array = (sol.y*N).astype('int')
    
    derivatives = np.ones((days,9))
    for t in range(days):
        derivatives[t] = solver.get_derivative(t=t, y=states_int_array[:,t])
    
    S_coeficeint=0
    E_coeficeint=0
    I_coeficeint=0.5
    R_mild_coeficeint=0.5
    R_severe_coeficeint=0.5
    R_severe_hosp_coeficeint=0.5
    R_R_fatal_coeficeint=1
    C_coeficeint=0
    D_coeficeint=1
    
    coeficeint= np.array([S_coeficeint,E_coeficeint,I_coeficeint,R_mild_coeficeint,R_severe_coeficeint,R_severe_hosp_coeficeint,\
                          R_R_fatal_coeficeint,C_coeficeint,D_coeficeint])
    
    grad1 = np.ones(days)
    grad2 = np.ones(days)

    for i in range(days):
        grad2[i] = np.dot(coeficeint,(derivatives[i]/1e8))
        grad1[i] = np.dot(coeficeint,(states_int_array[:,i]/1e8))
    return(grad1)

In [4]:
total_resource = 30
resource_spent = 0
window = 5
int_vec = np.ones(400)
while(resource_spent<total_resource):
    curve = run_seir(days=400, int_vec=int_vec)
    arg_curve = (-curve).argsort()
    day_max = arg_curve[0]
    i=1
    while(int_vec[day_max]>=2):
        day_max = arg_curve[i]
        i += 1

    int_vec[day_max] += 0.25 
    resource_spent += 0.25

In [5]:
int_vec

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [6]:
curve = run_seir(days=400, int_vec=int_vec)

In [7]:
np.sum(curve)

0.691228475

In [8]:
curve_base = run_seir(days = 400, int_vec=np.ones(400))
np.sum(curve_base)

0.8583847349999999